## VGG-16 모델로 합성곱 신경망 구현 하기

VGG-16 은 옥스포드 대학 연구팀에 의해 개발된 모델(VGGNet)로 2014년 이미지넷 이미지 인식대회(ILSVRC)에서 준우승을 했다. 

이전 모델은 대부분 얕은 층으로 구성된 네트워크를 사용했다면 VGG-16 이후로 층수가 높아지고 있다. 우승을 한 GoogleNet 보다 VGG-16 이 인기가 있는 이유는 간편함 때문이다. 

학습과 속도 관계로 VGG-16 모델을 줄여서 MNIST 학습을 진행해 본다.

Colab 의 [런타임] - [런타임 유형 변경] 을 클릭해서 [GPU]를 사용하도록 설정한다. 학습속도가 빨라진다.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# 데이터셋 준비 : MNIST

MNIST 를 가져온다. tensorflow 에서 기본으로 제공하는 데이터셋이므로 mnist.load_data() 를 사용한다.

In [ ]:
# TODO

Convolution Layer 를 사용해야 하므로 28x28 짜리 행렬이 아닌 채널이 추가된 텐서로 수정한다. reshape() 명령을 사용하면 된다.


In [ ]:
# TODO

((10000, 28, 28, 1), (60000, 28, 28, 1))

# VGG-16 의 합성곱 층(Convolution Layer) 만들기

Conv2D 와 MaxPooling2D 를 사용해서 합성층곱을 정의한다. 

CNN 은 배치크기를 제외하고 이미지의 높이, 너비, 채널 크기를 텐서입력으로 받는다. MNIST 는 흑백이므로 (28,28,1) 크기의 CNN 을 정의해서 사용한다. 

Cov(32, 3) - Pooling - Con(64, 3) - Pooling - Conv(64, 3) 으로 진행한다.

In [ ]:
model = models.Sequential()    # 
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1), padding='same'))  
# TODO





지금까지 만든 모델 구조를 확인한다. model.summary() 를 쓰면 모델의 구조를 확인할 수 있다.

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D 와 MaxPooling2D 출력은 (높이, 너비, 채널)의 텐서다. Conv2D 를 써서 채널을 늘리고, MaxPooling2D 를 써서 높이와 너비를 줄인다. 계산비용면에서 출력 채널을 늘리기 위해서는 높이와 너비를 줄여야한다.

# 전체 연결 Dense Layer

모델의 마지막은 Convolution Layer 의 출력 텐서를 2개의 전연결층(Dense)에 연결하고 분류를 진행한다. 

우선 Flatten() 을 사용해서 텐서를 벡터로 변환한다. 그 후 2개의 Dense() 층을 연결한다. 마지막은 softmax 를 사용해서 10개의 분류에 할당되도록 한다.

In [ ]:
# TODO




In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

7x7x64 텐서가 3136 벤터로 Flatten 되었다.

# 모델 컴파일

가장 보편적인 'adam' 과 크로스엔트로피, accuracy 를 사용해서 모델을 컴파일한다.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 훈련

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1192 - accuracy: 0.9638
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0394 - accuracy: 0.9878
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0268 - accuracy: 0.9915
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0205 - accuracy: 0.9930
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0152 - accuracy: 0.9949


# 평가

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0405 - accuracy: 0.9876


간단한 CNN 모델로 99%의 정확도를 얻었다.